In [2]:
def process_results(data):
    nested_values = ['video', 'author', 'music', 'stats', 'authorStats', 'challenges', 'duetInfo', 'textExtra', 'stickersOnItem']
    skip_values = ['challenges', 'duetInfo', 'textExtra', 'stickersOnItem']

    # Create blank dictionary
    flattened_data = {}
    # Loop through each video
    for idx, value in enumerate(data): 

        flattened_data[idx] = {}
        # Loop through each property in each video 
        for prop_idx, prop_value in value.items():
            # Check if nested
            if prop_idx in nested_values:
                if prop_idx in skip_values:
                    pass
                else:
                    # Loop through each nested property
                    for nested_idx, nested_value in prop_value.items():
                        flattened_data[idx][prop_idx+'_'+nested_idx] = nested_value
            # If it's not nested, add it back to the flattened dictionary
            else: 
                flattened_data[idx][prop_idx] = prop_value

    return flattened_data

In [3]:
from datetime import datetime
import os
import pandas as pd
import json

src_path = 'TikTok_scrape/output/'

count = 0
master_data = []

for root, dirs, files in os.walk(src_path):

    for filename in files:
        if "OrigSearch" in filename:
            # Load up data 
            with open(os.path.join(root, filename), 'r') as f:
                data = json.load(f)
                date = root.split("/")[2].replace("_fresh","")
                
                #data["scrape_date"] = datetime.strptime(date, '%d%m%Y')
                data["scrape_date"] = datetime.strptime(date, '%d%m%Y')
      
                master_data.append(data)
                
            f.close()
            
        count += 1


data_flat = process_results(master_data)
df = pd.DataFrame.from_dict(data_flat, orient='index')

df['convTime'] = df['createTime'].apply(datetime.fromtimestamp)
df['daysFromPost'] = df.scrape_date - df.convTime
df['scaledLikes'] = df.stats_diggCount/df.authorStats_heartCount

print(df.shape)
print(df.columns)

video_sub = df[['duetEnabled', 'stitchEnabled', 'shareEnabled', 'isAd', 'stats_shareCount',
       'stats_commentCount', 'stats_playCount', 'music_original', 'daysFromPost']]
author_sub = df[['authorStats_diggCount', 'authorStats_followingCount',
       'authorStats_followerCount', 'authorStats_heartCount',
       'authorStats_videoCount']]

all_sub = df[['authorStats_diggCount', 'authorStats_followingCount',
       'authorStats_followerCount', 'authorStats_heartCount',
       'authorStats_videoCount','duetEnabled', 'stitchEnabled', 'shareEnabled', 'isAd', 'stats_shareCount',
       'stats_commentCount', 'stats_playCount', 'music_original', 'daysFromPost']]
all_sub = all_sub.astype(int)

y = df[['stats_diggCount']]

(20401, 76)
Index(['id', 'desc', 'createTime', 'video_id', 'video_height', 'video_width',
       'video_duration', 'video_ratio', 'video_cover', 'video_originCover',
       'video_dynamicCover', 'video_playAddr', 'video_downloadAddr',
       'video_shareCover', 'video_reflowCover', 'video_bitrate',
       'video_encodedType', 'video_format', 'video_videoQuality',
       'video_encodeUserTag', 'author_id', 'author_uniqueId',
       'author_nickname', 'author_avatarThumb', 'author_avatarMedium',
       'author_avatarLarger', 'author_signature', 'author_verified',
       'author_secUid', 'author_secret', 'author_ftc', 'author_relation',
       'author_openFavorite', 'author_commentSetting', 'author_duetSetting',
       'author_stitchSetting', 'author_privateAccount', 'music_id',
       'music_title', 'music_playUrl', 'music_coverThumb', 'music_coverMedium',
       'music_coverLarge', 'music_authorName', 'music_original',
       'music_duration', 'music_album', 'stats_diggCount', 'stats_sh

In [61]:
from sklearn.preprocessing import StandardScaler
import numpy as np

# Sklearn
from sklearn.linear_model import LinearRegression # for building a linear regression model
from sklearn.svm import SVR # for building SVR model
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

# Visualizations
import plotly.graph_objects as go # for data visualization
import plotly.express as px # for data visualization

In [75]:
X = all_sub
print(X.shape)
print(y.shape)

(9931, 13)
(9931, 1)


In [76]:
sc_X = StandardScaler()
sc_y = StandardScaler()
X = sc_X.fit_transform(X)
y = sc_y.fit_transform(y)

In [ ]:
kf = KFold(n_splits=2)
kf.get_n_splits(X)

In [78]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# ------- Linear regression -------
model1 = LinearRegression()
lr = model1.fit(X_train, y_train)

# ------- Support Vector regression -------
model2 = SVR(kernel='rbf', C=1000, epsilon=5) # set kernel and hyperparameters
svr = model2.fit(X_train, y_train)

# # ------- Predict a range of values based on the models for visualization -------
# # Create 100 evenly spaced points from smallest X to largest X
# x_range = np.linspace(X.min(), X.max(), 100)

# # Predict y values for our set of X values
y_lr = model1.predict(X_test) # Linear regression
y_svr = model2.predict(X_test) # SVR

print(model1.score(X_test, y_test))
print(mean_squared_error(y_test, y_lr))
print(model2.score(X_test, y_test))
print(mean_squared_error(y_test, y_svr))

0.7842665887830752
0.20868362475392585
-18.436978537170987
18.80181244305511


/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
